### Algorithm 2
Create Mapping of Algo1's output to corresponding adjectives, adverbs, etc.

In [2]:
from statics import *
from pkgs import MongoClient, CoreNLPDependencyParser, nltk, re, defaultdict, stopwords
client = MongoClient(DB_CONNECTION)
dep_parser = CoreNLPDependencyParser(url=SERVER_URL)
negatives = list(stopwords.words())[1509:1644] # SLICE ENGLISH STOPWORDS ONLY


In [3]:
for entry in client.course_evals.derivatives.ngrams.filtered.find()[:5]:
    print(entry['course_id'],sorted(entry['filtered_ngrams'],key=lambda v:v[1],reverse=True)[:10])

W21-SURG-241-01/W21-ARTSTUDI-139-01 [['anatomy', 5], ['class', 5], ['classes', 4], ['drawing', 3], ['teacher', 3], ['muscles', 2], ['techniques', 2]]
W21-COMPMED-200-01 [['presentation', 2], ['professor', 2], ['questions', 2], ['skills', 1], ['answer', 1], ['club', 1], ['journal', 1], ['practice', 1]]
W21-CHPR-200-01 [['research', 9], ['guest', 4], ['prevention', 4], ['class', 3], ['work', 3], ['speakers', 2]]
W21-NENS-204-01 [['neurologist', 1], ['strokes', 1], ['topics', 1]]
W21-ORTHO-270-01 [['engineering', 2], ['research', 2], ['tissue', 2], ['advances', 1], ['cell', 1], ['stem', 1], ['class', 1], ['introduction', 1], ['unit', 1]]


In [4]:
def extract_nearby_qualitative_features(text, target):
    sent_tokenize = nltk.sent_tokenize(text)
    for s in sent_tokenize:
        pattern = re.compile('[^a-zA-z\s,.;:\']+', re.UNICODE)
        tmp = re.sub(pattern, ' ', s)
        tmp = re.sub('\s+', ' ', tmp).lower()
        if re.findall(target, tmp):
            yield list(dep_parser.parse(tmp.split()))[0]
    else: return []

In [7]:
#INSTRUCTOR NAMES
sub_names = set()
for entry in client.course_evals.entries.find():
    for sub_name in entry['INSTRUCTOR_NAME'].split(', '):
        if sub_name.lower() not in negatives:
            sub_names.add(sub_name.lower())
negatives.extend(list(sub_names))

### Algorithm 2

In [6]:
dep_grphs = defaultdict(lambda:defaultdict(list))
for row in client.course_evals.derivatives.ngrams.filtered.find()[:5]:
    print(row['course_id'])
    for top_feature in row['filtered_ngrams']:
        for entry in client.course_evals.entries.find({'COURSE_UNIQUE_ID':row['course_id']}):
            res = list(extract_nearby_qualitative_features(entry['RESPONSE_TEXT'],top_feature[0]))
            if len(res) > 0:
                dep_grphs[row['course_id']][top_feature[0]].extend(list(res))

W21-SURG-241-01/W21-ARTSTUDI-139-01
W21-COMPMED-200-01
W21-CHPR-200-01
W21-NENS-204-01
W21-ORTHO-270-01


In [8]:
related_words = defaultdict(lambda:defaultdict(list))
MAX_DEPTH = 2
def traverse_and_collect(addr,grph,depth,visited,target_class,target_feature):
    if depth == MAX_DEPTH: 
        return
    else:
        # ANCESTORS
        curr_word = grph.nodes[grph.nodes[addr]['head']]['word']
        if curr_word != target_feature and grph.nodes[addr]['head'] not in visited:
            if curr_word: related_words[target_class][target_feature].append((curr_word,depth))
            traverse_and_collect(grph.nodes[addr]['head'],grph,depth+1,visited+[addr],target_class,target_feature)
        # DESCENDANTS
        for _,addrs in grph.nodes[addr]['deps'].items():
            for a in addrs:
                dep_word = grph.nodes[a]['word']
                if dep_word not in negatives and grph.nodes[a]['rel'] != 'punct' and a not in visited:
                    if dep_word: related_words[target_class][target_feature].append((dep_word,depth))
                    traverse_and_collect(a,grph,depth+1,visited+[addr],target_class,target_feature)

for row in client.course_evals.derivatives.ngrams.filtered.find():
    for feature in dep_grphs[row['course_id']].keys():
        for grph in list(dep_grphs[row['course_id']][feature]):
            for n in list(grph.nodes):
                if grph.nodes[n]['word'] == feature:
                    traverse_and_collect(grph.nodes[n]['address'],grph,depth=0,visited=[],target_class=row['course_id'],target_feature=feature)

In [10]:
related_words['W21-SURG-241-01/W21-ARTSTUDI-139-01']

defaultdict(list,
            {'anatomy': [('facial', 0),
              ('muscles', 0),
              ('structure', 0),
              ('skeletal', 1),
              ('drawing', 0),
              ('techniques', 1),
              ('significant', 0),
              ('something', 1),
              ('learned', 0),
              ('underlying', 0),
              ('face', 0),
              ('neck', 1),
              ('learned', 0),
              ('area', 1),
              ('facial', 0),
              ('understand', 0),
              ('really', 1),
              ('easier', 1)],
             'class': [('flexible', 0),
              ('teacher', 1),
              ('extremely', 1),
              ('supportive', 1),
              ('enjoy', 0),
              ('learn', 1),
              ('immensely', 1),
              ('recommend', 0),
              ('really', 1),
              ('setting', 0),
              ('learn', 1),
              ('engaging', 1),
              ('engaging', 0),
              ('makes

### Save to DB

In [38]:
client.course_evals.derivatives.adjacent4.drop()
for key in related_words.keys():
    document = {
        'course_id': key,
        'feature_to_linkages': related_words[key]
    }
    client.course_evals.derivatives.adjacent4.insert_one(document)